In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

df = pd.read_excel(r'Megamillions.xlsx')

window_length = 7

df1 = df.copy()
df.drop(['Date'], axis=1, inplace=True)

number_of_features = df.shape[1]

train = df.copy()
train.head((window_length + 1))

train_rows = train.values.shape[0]
train_samples = np.empty([train_rows - window_length, window_length * number_of_features], dtype=float)
train_labels = np.empty([train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows - window_length):
    train_samples[i] = train.iloc[i: i + window_length].values.flatten()
    train_labels[i] = train.iloc[i + window_length: i + window_length + 1].values

scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

scaled_train_samples.head(window_length + 1)

x_train = np.empty([train_rows - window_length, window_length * number_of_features], dtype=float)
y_train = np.empty([train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows - window_length):
    x_train[i] = scaled_train_samples.iloc[i: i + window_length].values.flatten()
    y_train[i] = scaled_train_samples.iloc[i + window_length: i + window_length + 1].values

model = Sequential()
model.add(Dense(240, activation='relu', input_shape=(window_length * number_of_features,)))
model.add(Dropout(0.2))
model.add(Dense(240, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(240, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(240, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])

model.fit(x=x_train, y=y_train, batch_size=600, epochs=80000, verbose=2)

print('-' * 40)
print('Prediction vs. GroundTruth without rounding up or down')

for i in range(1, 10):
    test = df1.copy()
    test = test.tail(window_length + 10 - i)
    test = test.head(window_length + 1)
    test1 = test.drop(['Date'], axis=1)
    test1 = np.array(test1)
    x_test = scaler.transform(test1)
    x_test = x_test.flatten()[:window_length * number_of_features]  # Ajuste o tamanho dos dados de teste
    y_test_pred = model.predict(np.array([x_test]))
    y_test_true = test.drop(['Date'], axis=1).tail(1)
    print('Drawing Date', test.iloc[-1]['Date'])
    print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0] + 1)
    print('GroundTruth:\t', np.array(y_test_true)[0])
    print('-' * 40)

next_Date = '06/02/2023'

next = df.copy()
next = next.tail(window_length)
next = np.array(next)
x_next = scaler.transform(next)
x_next = x_next.flatten()[:window_length * number_of_features]  # Ajuste o tamanho dos dados de teste
y_next_pred = model.predict(np.array([x_next]))

print('-' * 40)
print('Predict the Future Drawing')
print('Drawing Date', next_Date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_next_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_next_pred).astype(int)[0] + 1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_next_pred).astype(int)[0] - 1)
print('-' * 40)


A saída de streaming foi truncada nas últimas 5000 linhas.
11/11 - 0s - loss: 0.0601 - accuracy: 0.8513 - 162ms/epoch - 15ms/step
Epoch 47529/50000
11/11 - 0s - loss: 0.0576 - accuracy: 0.8426 - 150ms/epoch - 14ms/step
Epoch 47530/50000
11/11 - 0s - loss: 0.0600 - accuracy: 0.8413 - 174ms/epoch - 16ms/step
Epoch 47531/50000
11/11 - 0s - loss: 0.0575 - accuracy: 0.8321 - 158ms/epoch - 14ms/step
Epoch 47532/50000
11/11 - 0s - loss: 0.0613 - accuracy: 0.8335 - 172ms/epoch - 16ms/step
Epoch 47533/50000
11/11 - 0s - loss: 0.0616 - accuracy: 0.8408 - 172ms/epoch - 16ms/step
Epoch 47534/50000
11/11 - 0s - loss: 0.0591 - accuracy: 0.8431 - 165ms/epoch - 15ms/step
Epoch 47535/50000
11/11 - 0s - loss: 0.0638 - accuracy: 0.8321 - 163ms/epoch - 15ms/step
Epoch 47536/50000
11/11 - 0s - loss: 0.0605 - accuracy: 0.8426 - 166ms/epoch - 15ms/step
Epoch 47537/50000
11/11 - 0s - loss: 0.0580 - accuracy: 0.8440 - 155ms/epoch - 14ms/step
Epoch 47538/50000
11/11 - 0s - loss: 0.0591 - accuracy: 0.8399 - 172m